In [16]:
from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score

In [61]:
dataframe = pd.read_csv("compas-scores-raw.csv")
dataframe = dataframe.iloc[:, :-3]

dfGroups = dataframe["Ethnic_Code_Text"].unique()

le = preprocessing.LabelEncoder()

y = dataframe.iloc[:, -3:]
y = y.iloc[:, -1:]
y[["ScoreText"]] = y[["ScoreText"]].apply(le.fit_transform)
x = dataframe.iloc[:, :-3]

for i in range(len(dfGroups)):
    x.loc[x["Ethnic_Code_Text"] == dfGroups[i], "Ethnic_Code_Text"] = i

x[["Agency_Text", "LastName", "FirstName", "MiddleName", "Sex_Code_Text", "DateOfBirth", "ScaleSet", "AssessmentReason", "Language", "LegalStatus", "CustodyStatus", "MaritalStatus", "Screening_Date", "RecSupervisionLevelText", "DisplayText"]] = x[["Agency_Text", "LastName", "FirstName", "MiddleName", "Sex_Code_Text", "DateOfBirth", "ScaleSet", "AssessmentReason", "Language", "LegalStatus", "CustodyStatus", "MaritalStatus", "Screening_Date", "RecSupervisionLevelText", "DisplayText"]].apply(le.fit_transform)
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=42)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(xTrain,yTrain)
predictions = clf.predict(xTest)

print(accuracy_score(yTest, predictions))
print("BIASED")
biasedResults = xTest.copy()
biasedResults["Real"] = yTest.to_numpy()
biasedResults["Pred"] = predictions
for i in range(len(dfGroups)):
    ethnicGroup = biasedResults[biasedResults["Ethnic_Code_Text"] == i]
    print(dfGroups[i])
    print(accuracy_score(ethnicGroup["Real"].tolist(), ethnicGroup["Pred"].tolist()))


eth = xTest["Ethnic_Code_Text"].tolist()
gen = xTest["Sex_Code_Text"].tolist()
age = xTest["DateOfBirth"].tolist()

xTrain = xTrain.drop(["Ethnic_Code_Text", "Sex_Code_Text", "DateOfBirth"], axis=1)
xTest = xTest.drop(["Ethnic_Code_Text", "Sex_Code_Text", "DateOfBirth"], axis=1)

clfu = tree.DecisionTreeClassifier()
clfu = clfu.fit(xTrain, yTrain)

predictionsu = clfu.predict(xTest)

print("UNBIASED")
xTest["Ethnic_Code_Text"] = eth
unResults = xTest.copy()
unResults["Real"] = yTest.to_numpy()
unResults["Pred"] = predictionsu
for i in range(len(dfGroups)):
    ethnicGroup = unResults[unResults["Ethnic_Code_Text"] == i]
    print(dfGroups[i])
    print(accuracy_score(ethnicGroup["Real"].tolist(), ethnicGroup["Pred"].tolist()))

0.7484591996055551
BIASED
Caucasian
0.7598469502588341
African-American
0.7146855756671021
Hispanic
0.7991803278688525
Other
0.8263358778625954
Asian
0.8095238095238095
African-Am
1.0
Native American
0.6136363636363636
Oriental
1.0
Arabic
0.8888888888888888
UNBIASED
Caucasian
0.7607472428539275
African-American
0.7098339242395969
Hispanic
0.7886416861826698
Other
0.8187022900763359
Asian
0.8412698412698413
African-Am
1.0
Native American
0.7954545454545454
Oriental
0.8
Arabic
0.8333333333333334


In [62]:
afr = dataframe[dataframe["Ethnic_Code_Text"] == "African-American"].to_numpy()
cau = dataframe[dataframe["Ethnic_Code_Text"] == "Caucasian"].to_numpy()
nat = dataframe[dataframe["Ethnic_Code_Text"] == "Native American"].to_numpy()

print("African-American: " + str(afr.shape[0]))
print("Caucasian: " + str(cau.shape[0]))
print("Native American: " + str(nat.shape[0]))


African-American: 27018
Caucasian: 21783
Native American: 219
